# Acesso SD

## Configuração do ambiente

### Na máquina local

Pré-requisito: distribuição Python (Anaconda ou Intel) instalada e funcionando na máquina local

* https://www.anaconda.com/products/individual
* https://software.intel.com/content/www/us/en/develop/tools/oneapi/components/distribution-for-python.html

#### Exemplo

Distribuição Intel (que inclui outros compiladores e ferramentas Intel)

* https://software.intel.com/content/www/us/en/develop/documentation/installation-guide-for-intel-oneapi-toolkits-linux/top/installation/install-using-package-managers/apt.html

*nix Ubuntu 20.04

```
...
sudo apt install intel-basekit intel-hpckit
sudo chmod -R 777 /opt/intel/oneapi/intelpython
source /opt/intel/oneapi/setvars.sh
conda install jupyterlab
jupyter-lab
```

### Exemplo de configuração do VPN e SSH

O SSH foi configurado para chave ao invés de senha

* https://github.com/efurlanm/hpc/blob/master/sd/access_configuring.ipynb
* https://www.hostinger.com.br/tutoriais/conexao-ssh-sem-senha

### Exemplo de configuração do JupyterLab

* https://github.com/efurlanm/hpc/blob/master/sd/access_using_jupyterlab.ipynb

### Nota:

Este Notebook usa o kernel bash :

    $ conda install -c conda-forge bash_kernel

Que necessita o comando a seguir (não tem a ver com o resto do texto):

In [20]:
bind 'set enable-bracketed-paste off'

## Acessando o Santos Dumont

* Primeiro passo: usando o Gerenciador de Rede, fazer a conexão ao VPN Cisco usando vpnc

Ao fazer login normalmente, uma máquina de login é selecionada aleatóriamente (geralmente sdumont11, sdumont13, ou sdumont14).

O comando a seguir abre uma conexão mestre para ser usada nos comandos subsequentes, e desta forma usamos sempre o mesmo nó de login:

In [27]:
export SSH_SOCKET=~/.ssh/remote_sd
ssh -MNfo ControlPath=$SSH_SOCKET sd
ssh -S $SSH_SOCKET sd 'hostname'

sdumont11


### Primeira sessão, no nó B-710

O comando `ssh` a seguir, executa os comandos no SD:

* carrega o módulo anaconda
* carraga o servidor jupyter-lab, que fica ativo na porta 20210

Uma instância do servidor jupyter-lab fica rodando na máquina de login

In [28]:
ssh -S $SSH_SOCKET sd 'module load anaconda3; jupyter-lab --no-browser --port=20210 --ip=0.0.0.0 > ~/20210.log 2>&1 & '

### Segunda sessão, no nó sequana (sdumont18)

* dentro da máquina de login chama o comando ssh para acessar o nó sequana (sdumont18) que foi configurado para usar chave ao invés de senha
* carrega uma nova instância do servidor jupyter-lab, que fica ativo no nó sdumont18 na porta 20211

In [29]:
ssh -S $SSH_SOCKET sd 'ssh -i ~/.ssh/id_sd_ed25519 sdumont18 "module load anaconda3; jupyter-lab --no-browser --port=20211 --ip=0.0.0.0 > ~/20211.log 2>&1 &" '

### Conexão das instâncias Jupyter à máquina local

Usando encaminhamento de portas

In [30]:
ssh -S $SSH_SOCKET sd -NTfL 8889:localhost:20210

In [31]:
ssh -S $SSH_SOCKET sd -NTfL 8890:sdumont18:20211

### Links para os clientes

In [32]:
ssh -S $SSH_SOCKET sd 'cat ~/20210.log | grep "     or" ' > ~/tk1.txt
temp=$(cat ~/tk1.txt)
echo ${temp/#*'20210'/'http://localhost:8889'}

http://localhost:8889/?token=76993a10b8285ff1325f51286ab20a501c95ef0d06641301


In [34]:
ssh -S $SSH_SOCKET sd 'cat ~/20211.log | grep "     or" ' > ~/tk2.txt
temp=$(cat ~/tk2.txt)
echo ${temp/#*'20211'/'http://localhost:8890'}

http://localhost:8890/?token=424e4cdced1cc74bfe5c57ae099d1b8827d9a106d8074c81


### Acesso aos arquivos usando o Gerenciador de Arquivos

É importante criar /pr e /scratch na máquina local, para evitar paths ou links quebrados.

In [97]:
sshfs sd:/prj     /prj     -o workaround=rename,uid=1000,gid=1000
sshfs sd:/scratch /scratch -o workaround=rename,uid=1000,gid=1000

## Sair

Uma forma é usando o menu do cliente Jupyter-Lab e selecionando "File > Shut Down" para cada instância do servidor, e em seguida encerrar a conexão mestre:

In [ ]:
ssh -S $SSH_SOCKET -O exit sd

Outra forma é matando os processos:

In [21]:
ssh -S $SSH_SOCKET sd 'pkill -u ${USER} -f jupyter-lab'

: 1

In [22]:
ssh -S $SSH_SOCKET sd 'ssh -i ~/.ssh/id_sd_ed25519 sdumont18 "pkill -u ${USER} -f jupyter-lab" '

In [23]:
ssh -S $SSH_SOCKET -O exit sd

Exit request sent.


Desmonta filesystem remoto, e encerra encaminhamento de portas:

In [24]:
sudo umount -fl /prj /scratch

umount: /prj: not mounted.
umount: /scratch: not mounted.


: 64

In [25]:
pkill -f 8889:

: 1

In [26]:
pkill -f 8890:

: 1